<a href="https://colab.research.google.com/github/LinReseach/LinReseach/blob/main/%E2%80%9CGaze360_Notebook_V2%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting gaze in videos

In this notebook, we will introduce the pipeline to compute gaze of people in videos. 

#Pose Detection with DensePose
The first step in our process is to use DensePose to detect people in the video.   

## Install DensePose through Detectron2

[Detectron2 installation](https://detectron2.readthedocs.io/en/latest/tutorials/install.html)

First, we are not going to install an Anaconda nor fix any dependencies:
https://colab.research.google.com/gist/simleo/bc9843b334090ffab2f5ca30922bbaf1/colabinstalltoolkit.ipynb#scrollTo=Sil5ydY5L2Q3

In [1]:
!pip install torch==1.11.0

Install Detectron2

In [ ]:
# Not needed
# !python -m pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

Install DensePose as package

In [2]:
!pip install git+https://github.com/facebookresearch/detectron2@main#subdirectory=projects/DensePose


  Cloning https://github.com/facebookresearch/detectron2 (to revision main) to /tmp/pip-req-build-0zhxgpcv
  Running command git clone -q https://github.com/facebookresearch/detectron2 /tmp/pip-req-build-0zhxgpcv
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-install-rs1tkvoj/detectron2_f977043f285840ebb51feba29fc4986c
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-install-rs1tkvoj/detectron2_f977043f285840ebb51feba29fc4986c
     |████████████████████████████████| 28.2 MB 1.7 MB/s 
     |████████████████████████████████| 47.8 MB 1.8 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 50 kB 5.2 MB/s 
     |████████████████████████████████| 74 kB 3.1 MB/s 
     |████████████████████████████████| 147 kB 49.6 MB/s 
     |████████████████████████████████| 130 kB 48.5 MB/s 
     |████████████████████████████████| 843 kB 42.2 MB/s 
     |████████████████████████████████| 7

Fix lot of imports.

In [3]:
!pip install imgaug==0.2.6
!pip uninstall opencv-python -y
!pip install opencv-python==4.1.2.30
import cv2

     |████████████████████████████████| 631 kB 5.3 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654017 sha256=a5fc030d6400239187115629bd4b6b36046086d983df07d6a97a36b35be981e3
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
Found existing installation: opencv-python 4.1.2.30
Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 28.3 MB 1.4 MB/s 


In [4]:
!pip uninstall tensorflow -y
!pip uninstall tensorflow-gpu -y
!pip install tensorflow==1.15
!pip install tensorflow-gpu==1.15

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0
     |████████████████████████████████| 412.3 MB 27 kB/s 
     |████████████████████████████████| 503 kB 45.4 MB/s 
     |████████████████████████████████| 3.8 MB 32.4 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=5428760a0b04f87dfc6019be3f5bae03c4a9e4aa2044852b9a37507115758ff7
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard

In [5]:
!pip install lucid==0.2.3
!pip install moviepy

     |████████████████████████████████| 40 kB 2.3 MB/s 
  Created wheel for lucid: filename=lucid-0.2.3-py3-none-any.whl size=59587 sha256=0b47cc8424c5afb084d2fefb2b87e70079c191af543dc5fef0682a03f480e083
  Stored in directory: /root/.cache/pip/wheels/f5/63/88/317069908f305e4a1e160ac72f2efb8e22a39b544c085e3bc0
Successfully built lucid


Download Detectron2 source.

In [6]:
!git clone https://github.com/facebookresearch/detectron2.git

Cloning into 'detectron2'...
remote: Enumerating objects: 14050, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 14050 (delta 0), reused 1 (delta 0), pack-reused 14042
Receiving objects: 100% (14050/14050), 5.78 MiB | 13.63 MiB/s, done.
Resolving deltas: 100% (10160/10160), done.


Check whether the DensePose installation was successful:

In [7]:
!python detectron2/tests/test_model_analysis.py

## Download video and run DensePose on it

We will now download the video, extract the frames and finally detect the pose of all the people in the video. 


Extract the frames of the video in a temporary folder:





In [8]:
!mkdir /content/temp
#!wget http://gaze360.csail.mit.edu/files/video.mp4
!ffmpeg -i video.mp4 /content/temp/frame%04d.jpg 
!rm -r /content/DensePose/DensePoseData/infer_out/*


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

Download model.

In [9]:
!cd detectron2/projects/DensePose && wget https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_101_FPN_s1x/165712084/model_final_c6ab63.pkl

--2022-05-04 08:40:11--  https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_101_FPN_s1x/165712084/model_final_c6ab63.pkl
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 331971130 (317M) [binary/octet-stream]
Saving to: ‘model_final_c6ab63.pkl’

model_final_c6ab63. 100%[===================>] 316.59M  33.0MB/s    in 9.7s    

2022-05-04 08:40:22 (32.5 MB/s) - ‘model_final_c6ab63.pkl’ saved [331971130/331971130]



Test model.

In [10]:
!cd detectron2/projects/DensePose && python apply_net.py show configs/densepose_rcnn_R_101_FPN_s1x.yaml model_final_c6ab63.pkl /content/temp/frame0001.jpg dp_contour,bbox --output image_densepose_contour.png

/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/detectron2/projects/DensePose/densepose/vis/densepose_results.py:133: UserWarning: The following kwargs were not used by contour: 'cfg', 'texture_atlas', 'texture_atlases_dict'
  plt.contour(u, self.levels, extent=extent, **self.plot_args)
/content/detectron2/projects/DensePose/densepose/vis/densepose_results.py:134: UserWarning: The following kwargs were not used by contour: 'cfg', 'texture_atlas', 'texture_atlases_dict'
  plt.contour(v, self.levels, extent=extent, **self.plot_args)


Run DensePose on the video frames:


In [11]:
!mkdir DensePoseData
!cd detectron2/projects/DensePose && python apply_net.py \
  dump \
  configs/densepose_rcnn_R_101_FPN_s1x.yaml model_final_c6ab63.pkl \
  /content/temp \
  -v \
  --output /content/DensePoseData/data.pkl

[05/04 08:40:38 apply_net]: Loading config from configs/densepose_rcnn_R_101_FPN_s1x.yaml
[05/04 08:40:38 apply_net]: Loading model from model_final_c6ab63.pkl
[05/04 08:40:42 apply_net]: Loading data from /content/temp
/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
[05/04 08:40:43 apply_net]: Processing /content/temp/frame0210.jpg
[05/04 08:40:44 apply_net]: Processing /content/temp/frame0077.jpg
[05/04 08:40:45 apply_net]: Processing /content/temp/frame0132.jpg
[05/04 08:40:46 apply_net]: Processing /content/temp/frame0240.jpg
[05/04 08:40:46 apply_net]: Processing /content/temp/frame0065.jpg
[05/04 08:40:47 apply_net]: Processing /content/temp/frame0108.jpg
[05/04 08:40:48 apply_net]: Processing /content/temp/frame0134.jpg

# Tracking people in the video
Although there are specific ways to track humans using DensePose, we will use a simpler procedure in this tutorial that could be used with any other head detector or pose estimator. Users are free to use their favorite tracking method to track people. 

We introduce some auxiliary functions to post-process the DensePose detections and track people on the video. 

## Processing DensePose results

We read the DensePose output and convert it into bounding box format:

In [12]:
import imageio
import cv2
import numpy as np
import os
from pathlib import Path
import pickle
import densepose

v_reader = imageio.get_reader('video.mp4')
v = list(v_reader)

with Path('./DensePoseData/data.pkl').open('rb') as fid:
  data = pickle.load(fid)

In [ ]:
#v.append(v[189])

In [ ]:
len(v)

NameError: ignored

In [13]:
with open('./DensePoseData/dataout.pkl', 'wb') as fid:
     pickle.dump(data, fid)

We do a simple tracking based on the intersection-over-union of the bounding boxes. Please plug here your preferred tracking algorithm. 

https://github.com/facebookresearch/detectron2/blob/main/projects/DensePose/doc/TOOL_APPLY_NET.md

---



In [14]:
def get_head_bbox(body_bbox, dense_pose_record):
    #Code from Detectron modifyied to make the box 30% bigger
    """Compute the tight bounding box of a binary mask."""
    # Head mask
    labels = dense_pose_record.labels.cpu().numpy()
    mask = (labels == 23) | (labels == 24)    
    if not np.any(mask):
      return None

    xs = np.where(np.sum(mask, axis=0) > 0)[0]
    ys = np.where(np.sum(mask, axis=1) > 0)[0]

    W = labels.shape[0]
    H = labels.shape[1]
    
    if len(xs) == 0 or len(ys) == 0:
        return None

    x0 = xs[0]
    x1 = xs[-1]

    y0 = ys[0]
    y1 = ys[-1]
    w = x1-x0
    h = y1-y0
    
    x0 = max(0,x0-w*0.15)
    x1 = max(0,x1+w*0.15)
    y0 = max(0,y0-h*0.15)
    y1 = max(0,y1+h*0.15)

    x0 += body_bbox[0]
    x1 += body_bbox[0]
    y0 += body_bbox[1]
    y1 += body_bbox[1]

    return np.array((x0, y0, x1, y1), dtype=np.float32)


def extract_heads_bbox(frame):
    N = len(frame['pred_densepose'])
    if N==0:
      return []
    bbox_list = []
    for i in range(N-1,-1,-1):
      dp_record = frame['pred_densepose'][i]
      body_bbox = frame['pred_boxes_XYXY'][i].cpu().numpy()
      bbox = get_head_bbox(body_bbox, dp_record)
      if bbox is None: 
        continue
      bbox_list.append(bbox)
    return bbox_list

names = np.array([int(Path(frame['file_name']).stem[-4:]) - 1 for frame in data], object)
bboxes = np.array([extract_heads_bbox(frame) for frame in data], object)
final_results = {n: b for n, b in zip(names, bboxes) if len(bboxes) > 0} 

#print(data[0]['pred_densepose'][0])
#print(data[0]['pred_boxes_XYXY'][0])
#print(bboxes[0][0])
#imageio.imwrite('test.png', (data[100]['pred_densepose'][1].labels.cpu().numpy() == 23).astype(float))

Try to track person ID over time by detecting overlap of bboxes using IoU.

In [44]:
def compute_iou(bb1,bb2):
    # determine the coordinates of the intersection rectangle
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1[2]-bb1[0]) * (bb1[3]-bb1[1])
    bb2_area = (bb2[2]-bb2[0]) * (bb2[3]-bb2[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    eps = 1e-8

    if iou <= 0.0  or iou > 1.0 + eps: return 0.0

    return iou
    
def find_id(bbox,id_dict):
    id_final = None
    max_iou = 0.5
    for k in id_dict.keys():
        if(compute_iou(bbox,id_dict[k][0])>max_iou): 
            id_final = k
            max_iou = compute_iou(bbox,id_dict[k][0])
    return id_final

# id_num = 0
# tracking_id = dict()
# identity_last = dict()
# frames_with_people = list(final_results.keys())

# frames_with_people.sort()
# for i in frames_with_people:
#     speople = final_results[i]
#     identity_next = dict()
#     for j in range(len(speople)):
#         bbox_head = speople[j]
#         
        
#         if bbox_head is None: continue
#         id_val = find_id(bbox_head,identity_last)
#         if id_val is None: 
#             id_num+=1
#             id_val = id_num
#         #TODO: Improve eye location
#         eyes = [(bbox_head[0]+bbox_head[2])/2.0, (0.65*bbox_head[1]+0.35*bbox_head[3])]
#         identity_next[id_val] = (bbox_head,eyes)
#     identity_last = identity_next
#     tracking_id[i] = identity_last
id_num = 0
tracking_id = dict()
identity_last = dict()
frames_with_people = list(final_results.keys())


frames_with_people.sort()
for i in frames_with_people:
    speople = final_results[i]
    identity_next = dict()
    bbox_size=dict()#rrrrrrrrrrrrrrrrrrrrrrr
    for j in range(len(speople)):
        bbox_head = speople[j]
        bbox_size[j]=abs(bbox_head[0]-bbox_head[2])+abs(bbox_head[1]-bbox_head[3])#rrrrrrrrrrrrrrrr
    maxj=max(bbox_size,key=bbox_size.get)#rrrrrrrrrrrrrrrrrrrrrrrrrr   
    bbox_head = speople[maxj]
    if bbox_head is None: continue
    id_val = find_id(bbox_head,identity_last)
    if id_val is None: 
        id_num+=1
        id_val = id_num
    #TODO: Improve eye location
    eyes = [(bbox_head[0]+bbox_head[2])/2.0, (0.65*bbox_head[1]+0.35*bbox_head[3])]
    identity_next[id_val] = (bbox_head,eyes)
    
    identity_last = identity_next
    tracking_id[i] = identity_last


In [42]:
speople[0]

array([488.33524, 831.4877 , 526.0352 , 881.4877 ], dtype=float32)

In [ ]:
bbox_size=abs(bbox_head[0]-bbox_head[2])+abs(bbox_head[1]-bbox_head[3])
len(speople)
id_num

46

# Gaze Estimation

Installing required packages and downloading Gaze360 code

In [46]:
import sys
import torch
import torchvision.transforms as transforms
import imageio
import cv2
import random
from PIL import Image
import math
import torch
import torchvision.transforms as transforms
import sys
image_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])


In [17]:
! git clone https://github.com/erkil1452/gaze360

Cloning into 'gaze360'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (91/91), done.


In [18]:
! wget http://gaze360.csail.mit.edu/files/gaze360_model.pth.tar

--2022-05-04 08:43:59--  http://gaze360.csail.mit.edu/files/gaze360_model.pth.tar
Resolving gaze360.csail.mit.edu (gaze360.csail.mit.edu)... 128.30.2.44
Connecting to gaze360.csail.mit.edu (gaze360.csail.mit.edu)|128.30.2.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58370849 (56M) [application/x-tar]
Saving to: ‘gaze360_model.pth.tar’

gaze360_model.pth.t 100%[===================>]  55.67M  11.6MB/s    in 5.3s    

2022-05-04 08:44:05 (10.5 MB/s) - ‘gaze360_model.pth.tar’ saved [58370849/58370849]



Loading the pre-trained model:

In [47]:
sys.path.append('/content/gaze360/code/')
from model import GazeLSTM
model = GazeLSTM()
model = torch.nn.DataParallel(model).cuda()
model.cuda()
checkpoint = torch.load('/content/gaze360_model.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
model.eval()

DataParallel(
  (module): GazeLSTM(
    (base_model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): B

OpenGL code required to draw the arrows in 3D. Part of the code used is from the [OpenGL Colaboratory](https://colab.research.google.com/github/tensorflow/lucid/blob/master/notebooks/differentiable-parameterizations/appendix/colab_gl.ipynb#scrollTo=Omj0FSZhLMXW) from the [TensorFlow Colab repository](https://colab.research.google.com/github/tensorflow/tensorflow/), which is registred by Google LLC with an [Apache Licence](https://www.apache.org/licenses/LICENSE-2.0). In compliance to that Licence, we want to make clear that the code was originally licenced by Google and in this repository we have modified it from the original source. 

If this fails, try again.

If this fails, Go tu Runtime->Change Runtime Type, select GPU and Confirm Save.

In [48]:
import numpy as np
import json
import moviepy.editor as mvp
from google.colab import files
import lucid.misc.io.showing as show
try:
  # This usually fails the first time.
  from lucid.misc.gl.glcontext import create_opengl_context
except:
  pass
from lucid.misc.gl.glcontext import create_opengl_context
import OpenGL.GL as gl
WIDTH, HEIGHT = 960, 720
create_opengl_context((WIDTH, HEIGHT))

gl.glClear(gl.GL_COLOR_BUFFER_BIT)


We define here the function to draw the arrow. The OpenGL code is also based on this [OpenGL code](https://www.shadertoy.com/view/3sjXWw). 

In [49]:
from OpenGL.GL import shaders
WIDTH, HEIGHT = 960, 720
create_opengl_context((WIDTH, HEIGHT))

vertexPositions = np.float32([[-1, -1], [1, -1], [-1, 1], [1, 1]])
VERTEX_SHADER = shaders.compileShader("""
#version 330
layout(location = 0) in vec4 position;
out vec2 UV;
void main()
{
  UV = position.xy*0.5+0.5;
  gl_Position = position;
}
""", gl.GL_VERTEX_SHADER)

FRAGMENT_SHADER = shaders.compileShader("""
#version 330
out vec4 outputColor;
in vec2 UV;

uniform sampler2D iChannel0;
uniform vec3 iResolution;
vec4 iMouse = vec4(0);
uniform float iTime = 0.0;
uniform float xpos = 100.0;
uniform float ypos = 0.1;
uniform float size = 10.0;

uniform float vdir_x = -1.0;
uniform float vdir_y = -1.0;
uniform float vdir_z = 1.0;



const vec3 X = vec3(1., 0., 0.);
const vec3 Y = vec3(0., 1., 0.);
const vec3 Z = vec3(0., 0., 1.);

// YOURS

const float Z_NEAR = 1.0;
const float Z_FAR  = 400.0;

const float EPSILON = 0.01;

const float FOCAL_LENGTH = 30.0;
const vec3 EYE_LOOK_POINT = vec3(0, 0, 5);

const vec3 WHITE = vec3(1, 1, 1);
const vec3 BLACK = vec3(0, 0, 0);
const vec3 RED =   vec3(1, 0, 0);
const vec3 GREEN = vec3(0, 1, 0);
const vec3 BLUE =  vec3(0, 0, 1);

const vec3 TOP_BG_COLOR = WHITE;
const vec3 BOT_BG_COLOR = GREEN;



const vec3 AMBIANT_COLOR = WHITE;
const vec3 SPECULAR_COLOR = WHITE;

const float AMBIANT_RATIO = 0.3;
const float DIFFUSE_RATIO = 0.8;
const float SPECULAR_RATIO = 0.4;
const float SPECULAR_ALPHA = 5.;

const vec3 LIGHT_DIRECTION = normalize(vec3(1, -1, -1));

vec2 normalizeAndCenter(in vec2 coord) {
    return (2.0 * coord - iResolution.xy) / iResolution.y;
}

vec3 rayDirection(vec3 eye, vec2 uv) {    
    vec3 z = normalize(eye - EYE_LOOK_POINT);
    vec3 x = normalize(cross(Y, z));
    vec3 y = cross(z, x);
    
    return normalize(
          x * uv.x 
        + y * uv.y 
        - z * FOCAL_LENGTH);
}

//
// Rotations
//

vec3 rotX(vec3 point, float angle) {
    mat3 matRotX = mat3(
        1.0, 0.0, 0.0, 
        0.0, cos(angle), -sin(angle), 
        0.0, sin(angle), cos(angle));
    return matRotX * point;
}

vec3 rotY(vec3 point, float angle) {
    mat3 matRotY = mat3( 
        cos(angle*0.5), 0.0, -sin(angle*0.5),
        0.0, 1.0, 0.0, 
        sin(angle*0.5), 0.0, cos(angle*0.5));
    return matRotY * point;
}

vec3 rotZ(vec3 point, float angle) {
    mat3 matRotZ = mat3(
        cos(angle*0.1), -sin(angle*0.1), 0.0, 
        sin(angle*0.1), cos(angle*0.1), 0.0,
    	0.0, 0.0, 1.0);
    return matRotZ * point;
}

//
// Positioning
//

vec3 randomOrtho(vec3 v) {
    if (v.x != 0. || v.y != 0.) {
    	return normalize(vec3(v.y, -v.x, 0.));
    } else {
    	return normalize(vec3(0., v.z, -v.y));
    }
} 

vec3 atPosition(vec3 point, vec3 position) {
	return (point - position);
}

vec3 atCoordSystem(vec3 point, vec3 center, vec3 dx, vec3 dy, vec3 dz) {
	vec3 localPoint = (point - center);
    return vec3(
        dot(localPoint, dx),
        dot(localPoint, dy),
        dot(localPoint, dz));
}

vec3 atCoordSystemX(vec3 point, vec3 center, vec3 dx) {
    vec3 dy = randomOrtho(dx);
    vec3 dz = cross(dx, dy);
    
    return atCoordSystem(point, center, dx, dy, dz);
}

vec3 atCoordSystemY(vec3 point, vec3 center, vec3 dy) {
    vec3 dz = randomOrtho(dy);
    vec3 dx = cross(dy, dz);
    
    return atCoordSystem(point, center, dx, dy, dz);
}

vec3 atCoordSystemZ(vec3 point, vec3 center, vec3 dz) {
    vec3 dx = randomOrtho(dz);
    vec3 dy = cross(dz, dx);
    
    return atCoordSystem(point, center, dx, dy, dz);
}

//
// Shapes
//

float capsule(vec3 coord, float height, float radius)
{
    coord.y -= clamp( coord.y, 0.0, height );
    return length( coord ) - radius;
}

float roundCone(vec3 coord, in float radiusTop, float radiusBot, float height)
{
    vec2 q = vec2( length(coord.xz), coord.y );
    
    float b = (radiusBot-radiusTop)/height;
    float a = sqrt(1.0-b*b);
    float k = dot(q,vec2(-b,a));
    
    if( k < 0.0 ) return length(q) - radiusBot;
    if( k > a*height ) return length(q-vec2(0.0,height)) - radiusTop;
        
    return dot(q, vec2(a,b) ) - radiusBot;
}

//
// Boolean ops
//

vec4 shape(float dist, vec3 color) {
	return vec4(color, dist);
}

vec4 join(vec4 shape1, vec4 shape2) {
    if (shape1.a < shape2.a) {
    	return shape1;
    } else {
    	return shape2;
    }
}

vec4 join(vec4 shape1, vec4 shape2, vec4 shape3) {
    return join(join(shape1, shape2), shape3);
}

vec4 join(vec4 shape1, vec4 shape2, vec4 shape3, vec4 shape4) {
    return join(join(shape1, shape2, shape3), shape4);
}



//
// Scene
// x range: 355
// y rangeL 205



vec4 dist(in vec3 coord) {
    vec3 V_y = normalize(vec3(vdir_x,vdir_y,vdir_z));
    
    vec3 V_x = randomOrtho(V_y);
        
    vec3 V_z = cross(V_x,V_y);
    
    vec3 pos = vec3(xpos,ypos,0);
        
    vec3 offset = pos+V_y*size*4.0;
    vec3 ARM_2_X = V_x;
    vec3 ARM_2_Y = V_y;
    vec3 ARM_2_Z = V_z;
    vec3 NOZE_O = vec3(0,11,0);
    vec3 NOZE_X = -V_x;
    vec3 NOZE_Y = V_y;
    vec3 NOZE_Z = V_z;

    vec4 noze = shape(roundCone(atCoordSystem(coord, offset, NOZE_X, NOZE_Y, NOZE_Z), size*0.05, size*0.4, size*2.0), RED);
    vec4 rightArm = shape(capsule(atCoordSystem(coord, pos, ARM_2_X, ARM_2_Y, ARM_2_Z), size*5., size*0.1), RED);
    return join(noze, rightArm);
}

//
//
//

bool rayMarching(in vec3 startPoint, in vec3 direction, out vec3 lastPoint, out vec3 color) {
    lastPoint = startPoint;
    for (int i = 0; i < 50; ++i) {
        vec4 d = dist(lastPoint);
        if (d.a < EPSILON) {
            color = d.xyz;
            return true;
        } else {
            lastPoint += d.a * direction;
        }
        if (lastPoint.z < -Z_FAR) {
            break;
        }
    }    
    return false;
}

vec3 norm(in vec3 coord) {
	vec3 eps = vec3( EPSILON, 0.0, 0.0 );
	vec3 nor = vec3(
	    dist(coord+eps.xyy).a - dist(coord-eps.xyy).a,
	    dist(coord+eps.yxy).a - dist(coord-eps.yxy).a,
	    dist(coord+eps.yyx).a - dist(coord-eps.yyx).a);
	return normalize(nor);
}


vec3 cellShadingObjColor(vec3 point, vec3 ray, vec3 objColor) {
    vec3 n = norm(point);
    
    float diffuseValue = max(dot(-LIGHT_DIRECTION, n), 0.);
    float specularValue = pow(max(dot(-reflect(LIGHT_DIRECTION, n), ray), 0.), SPECULAR_ALPHA);        
    return AMBIANT_COLOR * AMBIANT_RATIO
        + objColor * DIFFUSE_RATIO * diffuseValue
        + SPECULAR_COLOR * SPECULAR_RATIO * specularValue;
}


vec3 computeColor(vec2 fragCoord) {
    vec2 uv = normalizeAndCenter(fragCoord);
    vec3 eye = vec3(0, 0, 20);
        
    vec3 ray = rayDirection(eye, uv);
    
    vec3 intersection;
    vec3 color;
    bool intersected = rayMarching(eye, ray, intersection, color);
    if (intersected) {
    	return cellShadingObjColor(intersection, ray, color);
    } else {
    	return vec3(0,0,0);
    }
}

//#define SUPERSAMPLING
void mainImage( out vec4 fragColor, in vec2 fragCoord )
{    
    #ifdef SUPERSAMPLING
    fragColor = vec4(0);
    float count = 0.;
    for(float dx=-0.5; dx<=0.5; dx+=0.5) {
    	for(float dy=-0.5; dy<=0.5; dy+=0.5) {
            fragColor += vec4(computeColor(fragCoord + vec2(dx, dy)), 1.0);
            count += 1.;
        }
    }
    
    fragColor /= count;
    
    #else
    fragColor = vec4(computeColor(fragCoord),1.0);
    #endif
} 



void main()
{
    mainImage(outputColor, UV*iResolution.xy);
}

""", gl.GL_FRAGMENT_SHADER)

shader = shaders.compileProgram(VERTEX_SHADER, FRAGMENT_SHADER)

xpos = gl.glGetUniformLocation(shader, 'xpos')
ypos = gl.glGetUniformLocation(shader, 'ypos')

vdir_x = gl.glGetUniformLocation(shader, 'vdir_x')
vdir_y = gl.glGetUniformLocation(shader, 'vdir_y')
vdir_z = gl.glGetUniformLocation(shader, 'vdir_z')

arrow_size = gl.glGetUniformLocation(shader, 'size')

res_loc = gl.glGetUniformLocation(shader, 'iResolution')

def render_frame(x_position,y_position,vx,vy,vz,asize):
  gl.glClear(gl.GL_COLOR_BUFFER_BIT)
  with shader:

    x_position = x_position*0.89
    y_position = y_position*0.67
    gl.glUniform1f(xpos, x_position)
    gl.glUniform1f(ypos, y_position)

    gl.glUniform1f(vdir_x, vx)
    gl.glUniform1f(vdir_y, vy)
    gl.glUniform1f(vdir_z, vz)
    gl.glUniform1f(arrow_size, asize)

    gl.glUniform3f(res_loc, WIDTH, HEIGHT, 1.0)
    
    gl.glEnableVertexAttribArray(0);
    gl.glVertexAttribPointer(0, 2, gl.GL_FLOAT, False, 0, vertexPositions)
    gl.glDrawArrays(gl.GL_TRIANGLE_STRIP, 0, 4)
  img_buf = gl.glReadPixels(0, 0, WIDTH, HEIGHT, gl.GL_RGB, gl.GL_UNSIGNED_BYTE)
  img = np.frombuffer(img_buf, np.uint8).reshape(HEIGHT, WIDTH, 3)[::-1]
  return img


Defining the necessary functions and installing packages to predict the 3D gaze and draw it in the image:



In [50]:

def spherical2cartesial(x): 
    output = torch.zeros(x.size(0),3)
    output[:,2] = -torch.cos(x[:,1])*torch.cos(x[:,0])
    output[:,0] = torch.cos(x[:,1])*torch.sin(x[:,0])
    output[:,1] = torch.sin(x[:,1])
    return output


In [ ]:
tracking_id[26]


{1: (array([489.37875, 834.4072 , 525.77875, 885.1072 ], dtype=float32),
  [507.5787353515625, 852.1522094726563]),
 2: (array([ 919.3035 ,  256.43073, 1176.7035 ,  595.58075], dtype=float32),
  [1048.0035400390625, 375.13323364257815])}

Predict gaze and generate the final video:

In [51]:
fps = v_reader.get_meta_data()['fps']
out = imageio.get_writer('video_gaze_imageio.mp4',fps=fps)
color_encoding = []
for i in range(1000): color_encoding.append([random.randint(0,254),random.randint(0,254),random.randint(0,254)])

W = max(int(fps//8),1)
gaze_array=[]
eye_array=[]
for i in range(0,len(v)):
    print(i,len(v))
    image = v[i].copy()
    image = cv2.resize(image,(WIDTH,HEIGHT))
    image = image.astype(float)
    gaze_list=[]
    eye_list=[]
    #if i in tracking_id.keys():
    if i in tracking_id.keys():
        for id_t in tracking_id[i].keys():
            input_image = torch.zeros(7,3,224,224)
            count = 0
            for j in range(i-3*W,i+4*W,W):
                if j in tracking_id and id_t in tracking_id[j]:
                    new_im = Image.fromarray(v[j],'RGB')
                    bbox,eyes = tracking_id[j][id_t]
                else:
                    new_im = Image.fromarray(v[i],'RGB')
                    bbox,eyes = tracking_id[i][id_t]
                new_im = new_im.crop((bbox[0],bbox[1],bbox[2],bbox[3]))
                input_image[count,:,:,:] = image_normalize(transforms.ToTensor()(transforms.Resize((224,224))(new_im)))
                count = count+1
            
            bbox,eyes = tracking_id[i][id_t] 
            bbox = np.asarray(bbox).astype(int)

            # imageio.imwrite('temp.jpg', input_image[0].permute(1,2,0).cpu().numpy())

            output_gaze,_ = model(input_image.view(1,7,3,224,224).cuda())
            gaze = spherical2cartesial(output_gaze).detach().numpy()
            eyes = np.asarray(eyes).astype(float)
            eyes[0],eyes[1] = eyes[0]/float(v[i].shape[1]),eyes[1]/float( v[i].shape[0])

            gaze = gaze.reshape((-1))
            gaze_list.append(gaze)
            eye_list.append(eyes)
            img_arrow = render_frame(2*eyes[0]-1,-2*eyes[1]+1,-gaze[0],gaze[1],-gaze[2],0.05)
            binary_img = ((img_arrow[:,:,0]+img_arrow[:,:,1]+img_arrow[:,:,2])==0.0).astype(float)
            binary_img = np.reshape(binary_img,(HEIGHT,WIDTH,1))
            binary_img = np.concatenate((binary_img,binary_img,binary_img), axis=2)
            image = binary_img*image + img_arrow*(1-binary_img)
            image = image.astype(np.uint8)
            bbox[0],bbox[2] = WIDTH*bbox[0]/v[i].shape[1],WIDTH*bbox[2]/v[i].shape[1]
            bbox[1],bbox[3] = HEIGHT*bbox[1]/v[i].shape[0],HEIGHT*bbox[3]/v[i].shape[0]
            image = cv2.rectangle(image, (bbox[0],bbox[1]), (bbox[2],bbox[3]),color_encoding[min(id_t,900)])
            image = image.astype(float)
    gaze_array.append(gaze_list)
    eye_array.append(eye_list)
    image = image.astype(np.uint8)
    out.append_data(image)
out.close()


0 243
1 243
2 243
3 243
4 243
5 243
6 243
7 243
8 243
9 243
10 243
11 243
12 243
13 243
14 243
15 243
16 243
17 243
18 243
19 243
20 243
21 243
22 243
23 243
24 243
25 243
26 243
27 243
28 243
29 243
30 243
31 243
32 243
33 243
34 243
35 243
36 243
37 243
38 243
39 243
40 243
41 243
42 243
43 243
44 243
45 243
46 243
47 243
48 243
49 243
50 243
51 243
52 243
53 243
54 243
55 243
56 243
57 243
58 243
59 243
60 243
61 243
62 243
63 243
64 243
65 243
66 243
67 243
68 243
69 243
70 243
71 243
72 243
73 243
74 243
75 243
76 243
77 243
78 243
79 243
80 243
81 243
82 243
83 243
84 243
85 243
86 243
87 243
88 243
89 243
90 243
91 243
92 243
93 243
94 243
95 243
96 243
97 243
98 243
99 243
100 243
101 243
102 243
103 243
104 243
105 243
106 243
107 243
108 243
109 243
110 243
111 243
112 243
113 243
114 243
115 243
116 243
117 243
118 243
119 243
120 243
121 243
122 243
123 243
124 243
125 243
126 243
127 243
128 243
129 243
130 243
131 243
132 243
133 243
134 243
135 243
136 243
137 243
138 24

Finally, we play the video with the gaze visualization: 

In [52]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('video_gaze_imageio.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


In [53]:
with open('./DensePoseData/gaze.pkl', 'wb') as fid:
     pickle.dump(gaze_array, fid)

In [ ]:
with open('./DensePoseData/eye.pkl', 'wb') as fid:
     pickle.dump(eye_array, fid)